In [7]:
import requests
import pandas as pd
from data_processor import TennisDataProcessor

pd.options.display.max_columns = None
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/refs/heads/master/atp_matches_{}.csv"
data_20s = pd.concat([pd.read_csv(url.format(year)) for year in range(2000, 2025)])


In [8]:
tdp = TennisDataProcessor(data_20s.copy())
derived_data = tdp.derive_match_data()